# Case 1: Binomial Trees

Asha de Meij (i6254733)

**Task:** Consider the unit-linked contract with maturity guarantee that was discussed in the lecture.
Take the risk-free rate equal to r = 1%, the volatility of the investment fund o = 15% and
the final guaranteed amount at T = 10 equal to 100 (i.e., we have a contract with a 0%
guarantee).

We want to use a binomial tree with increasing number of steps to compute the price of this
contract. In the lecture we used 4 time-steps, but this is not realistic. Make a computer
program where you increase the number of steps in the binomial tree from 4 to 100 and
make a graph of how the price of the contract as a function of the number of steps develops.

**Questions:**
- Can you explain the pattern that you see in the graph? To help you investigate the
convergence pattern, make an extra plot of the stock price S, in two nearest nodes
at T = 10, just above and below the guarantee level, as a function of the number of
steps.
- When you keep the number of steps fixed (say 100), how does the price of the
contract change if you change r or o or T. Can you explain these changes?
- Can you modify your program to make it run faster (e.g., for a tree with 10 000
steps)? If so, what changes did you make compared to your initial code? Describe
how you have analysed the run-time of the code.

# Code

In [1]:
print("hi")

hi
